In [11]:
import json
import re

def extract_qa_pairs(message):
    # Regex tìm các đoạn chứa Câu hỏi và Trả lời
    pattern = r"Câu hỏi:\n(.*?)\n===\nTrả lời:\n(.*?)(?=\n===\nCâu hỏi:|\Z)"
    matches = re.findall(pattern, message, re.DOTALL)
    
    # Xử lý danh sách kết quả để loại bỏ khoảng trắng thừa
    qa_pairs = [(q.strip(), a.strip()) for q, a in matches]
    
    return qa_pairs


In [12]:
with open("sample_check.json", "r") as f:
    data = json.load(f)

In [13]:
data

{'messages': 'Câu hỏi:\nBức ảnh này chụp bộ phận nào?\n===\nTrả lời:\nBức ảnh này chụp toàn bộ cơ thể của bệnh nhân.\n===\nCâu hỏi:\nCó phát hiện bất thường nào ở hệ thống xương không?\n===\nTrả lời:\nCó, hình ảnh cho thấy một nốt đặc xương đường kính 11 mm ở chỏm xương đùi trái. Tuy nhiên, nốt này không tăng chuyển hóa FDG, điều này gợi ý rằng khả năng cao là tổn thương lành tính.\n===\nCâu hỏi:\nCó tổn thương nào nghi ngờ ác tính ở hệ thống cơ xương không?\n===\nTrả lời:\nKhông, hình ảnh không ghi nhận bất thường nào có tăng chuyển hóa FDG trong hệ thống cơ xương. Điều này có nghĩa là chưa có dấu hiệu gợi ý tổn thương ác tính hoặc di căn xương trên phim chụp.\n===\nCâu hỏi:\nNhìn chung, có dấu hiệu nào gợi ý tổn thương ác tính trong hệ thống cơ xương không?\n===\nTrả lời:\nDựa trên hình ảnh, không có dấu hiệu rõ ràng của tổn thương á'}

In [14]:
qa_pairs = extract_qa_pairs(data["messages"])

# Print results
for i, (q, a) in enumerate(qa_pairs, 1):
    print(f"Q{i}: {q}")
    print(f"A{i}: {a}")
    print("-" * 50)


Q1: Bức ảnh này chụp bộ phận nào?
A1: Bức ảnh này chụp toàn bộ cơ thể của bệnh nhân.
--------------------------------------------------
Q2: Có phát hiện bất thường nào ở hệ thống xương không?
A2: Có, hình ảnh cho thấy một nốt đặc xương đường kính 11 mm ở chỏm xương đùi trái. Tuy nhiên, nốt này không tăng chuyển hóa FDG, điều này gợi ý rằng khả năng cao là tổn thương lành tính.
--------------------------------------------------
Q3: Có tổn thương nào nghi ngờ ác tính ở hệ thống cơ xương không?
A3: Không, hình ảnh không ghi nhận bất thường nào có tăng chuyển hóa FDG trong hệ thống cơ xương. Điều này có nghĩa là chưa có dấu hiệu gợi ý tổn thương ác tính hoặc di căn xương trên phim chụp.
--------------------------------------------------
Q4: Nhìn chung, có dấu hiệu nào gợi ý tổn thương ác tính trong hệ thống cơ xương không?
A4: Dựa trên hình ảnh, không có dấu hiệu rõ ràng của tổn thương á
--------------------------------------------------


In [3]:
import os
import json
import hashlib
import re

def convert_json(old_data, seq):
    """
    Chuyển đổi một đối tượng JSON theo định dạng:
    {
        "id": "<seq>_<md5(report_id)>",
        "image": "<tên file, với report_id thay '/reports/' thành '/images/' và đuôi '.txt' thành '.npy'>",
        "conversations": [
            {"from": "human", "value": "<question>"},
            {"from": "gpt", "value": "<answer>"},
            ...
        ]
    }
    Nếu phần tử không đúng định dạng (ví dụ, trường 'qa' không có các cặp "Câu hỏi:" - "Trả lời:"), ném ngoại lệ.
    """
    if "report_id" not in old_data or "qa" not in old_data:
        raise ValueError("Thiếu trường bắt buộc 'report_id' hoặc 'qa'")
    
    report_id = old_data["report_id"]
    # Tạo hash MD5 từ report_id
    hash_str = hashlib.md5(report_id.encode("utf-8")).hexdigest()
    new_id = f"{seq}_{hash_str}"
    
    # Chuyển đổi đường dẫn file: thay '/reports/' thành '/images/' và đổi đuôi .txt thành .npy
    image_path = report_id.replace("/reports/", "/images/")
    if image_path.endswith(".txt"):
        image_path = image_path[:-4] + ".npy"
    image_file = image_path
    
    # Xử lý trường "qa": Tách các cặp "Câu hỏi:" - "Trả lời:" bằng regex
    qa_text = old_data["qa"]
    conversations = []
    pattern = re.compile(
        r"Câu hỏi:\s*(.*?)\s*===\s*Trả lời:\s*(.*?)(?=\n*Câu hỏi:|$)",
        re.DOTALL
    )
    matches = re.findall(pattern, qa_text)
    if not matches:
        raise ValueError("Trường 'qa' không đúng định dạng mong đợi.")
    
    for question, answer in matches:
        conversations.append({
            "from": "human",
            "value": question.strip()
        })
        conversations.append({
            "from": "gpt",
            "value": answer.strip()
        })
    
    new_data = {
        "id": new_id,
        "image": image_file,
        "conversations": conversations
    }
    return new_data

def process_jsonl_file(input_file, output_file, error_file):
    """
    Đọc file JSONL chứa nhiều đối tượng JSON (mỗi dòng một đối tượng),
    chuyển đổi từng đối tượng theo định dạng mới.
    Nếu một phần tử không đúng định dạng (ví dụ, trường 'qa' không hợp lệ),
    ghi report_id của phần tử đó vào file error_file.
    Kết quả chuyển đổi được lưu vào output_file (file JSON chứa danh sách các đối tượng).
    """
    converted_list = []
    error_ids = []
    seq = 1

    with open(input_file, "r", encoding="utf-8") as fin:
        for line_number, line in enumerate(fin, start=1):
            line = line.strip()
            if not line:
                continue
            try:
                element = json.loads(line)
                converted = convert_json(element, seq)
                converted_list.append(converted)
                seq += 1
            except Exception as e:
                # Nếu không thể chuyển đổi, lưu report_id (nếu có) vào danh sách lỗi
                try:
                    element = json.loads(line)
                    report_id = element.get("report_id", f"unknown_line_{line_number}")
                except Exception:
                    report_id = f"unknown_line_{line_number}"
                print(f"Lỗi xử lý phần tử tại dòng {line_number} với report_id '{report_id}': {e}")
                error_ids.append(report_id)
    
    # Lưu kết quả chuyển đổi ra file JSON (danh sách các đối tượng)
    with open(output_file, "w", encoding="utf-8") as fout:
        json.dump(converted_list, fout, ensure_ascii=False, indent=2)
    
    # Lưu report_id của các phần tử lỗi vào file txt
    with open(error_file, "w", encoding="utf-8") as ferr:
        for rid in error_ids:
            ferr.write(rid + "\n")

if __name__ == "__main__":
    input_file = "conversations.jsonl"             # File JSONL chứa nhiều đối tượng JSON (mỗi dòng một đối tượng)
    output_file = "converted_output.json"  # File JSON chứa các phần tử đã chuyển đổi (danh sách các đối tượng)
    error_file = "error_report_ids.txt"      # File txt chứa report_id của các phần tử lỗi

    process_jsonl_file(input_file, output_file, error_file)

Lỗi xử lý phần tử tại dòng 9 với report_id '/home/user01/aiotlab/ducntm/DATA/FmMedical/THANG 3/reports/head_neck/day_16_patient_185.txt': Trường 'qa' không đúng định dạng mong đợi.
Lỗi xử lý phần tử tại dòng 11 với report_id '/home/user01/aiotlab/ducntm/DATA/FmMedical/THANG 3/reports/head_neck/day_16_patient_182.txt': Trường 'qa' không đúng định dạng mong đợi.
Lỗi xử lý phần tử tại dòng 31 với report_id '/home/user01/aiotlab/ducntm/DATA/FmMedical/THANG 3/reports/head_neck/day_10_patient_150.txt': Trường 'qa' không đúng định dạng mong đợi.
Lỗi xử lý phần tử tại dòng 51 với report_id '/home/user01/aiotlab/ducntm/DATA/FmMedical/THANG 3/reports/head_neck/day_14_patient_158.txt': Trường 'qa' không đúng định dạng mong đợi.
Lỗi xử lý phần tử tại dòng 117 với report_id '/home/user01/aiotlab/ducntm/DATA/FmMedical/THANG 3/reports/head_neck/day_3_patient_272.txt': Trường 'qa' không đúng định dạng mong đợi.
Lỗi xử lý phần tử tại dòng 144 với report_id '/home/user01/aiotlab/ducntm/DATA/FmMedical/TH